In [19]:
import pandas as pd
import numpy as np
import datetime
from scipy.optimize import minimize

In [20]:
def call_payoff(s, k):
    return max(s - k, 0)

def exposure(s1, s0, k):
    return call_payoff(s1, k) - call_payoff(s0, k) 

def find_value(multiplier_list, date_diff, price, exposure_value):
    """
    Look up the appropriate multiplier given a date difference.
    Since the contract’s DateDiff is 1-based (minimum 1) and the passed list
    is expected to be of length equal to the maximum DateDiff, we subtract one
    from date_diff. If date_diff exceeds the list length, we use the last element.
    """
    index = max(date_diff - 1, 0)
    index = min(index, len(multiplier_list) - 1)
    return multiplier_list[index] * exposure_value / price

In [51]:
def setup():
    def load_data(filepath):
        return pd.read_csv(filepath)

    # df = load_data('BADSS training data.csv')
    df = load_data('BADSS test data.csv')
    # Fix column names with extra spaces.
    df = df.rename(columns={'Bid Size ': 'Bid Size', 'Ask Size ': 'Ask Size'})

    # Convert to datetime and then map to trading day indices
    df['Date1'] = pd.to_datetime(df['Date'])
    df['Maturity1'] = pd.to_datetime(df['Maturity'])
    unique_trading_days = sorted(set(df['Date1']).union(set(df['Maturity1'])))
    trading_day_to_index = {d: i + 1 for i, d in enumerate(unique_trading_days)}

    df['Date1'] = df['Date'].map(trading_day_to_index)
    df['Maturity1'] = df['Maturity'].map(trading_day_to_index)

    df.drop(columns=['Date', 'Maturity'], inplace=True)
    df['DateDiff'] = df['Maturity1'] - df['Date1']

    df.rename(columns={"Date1": "Date"}, inplace=True)
    df.rename(columns={"Maturity1": "Maturity"}, inplace=True)

    return df

In [53]:
def loss_finder(multiplier_list):
    """
    Computes the total running cost for maintaining an exposure target by processing
    option contract data day by day. Now, in addition to buying contracts to reach
    the target exposure, we sell (and realize money) every day where a bid price is available.
    
    multiplier_list: list of multipliers (expected length equal to maximum DateDiff)
    """
    df = setup()
    # Define symbol-specific multipliers for underlying price moves.
    symbol_multipliers = {'SPY': 1.03, 'QQQ': 1.036, 'IWM': 1.041}

    # Calculate initial exposure per option row.
    def calculate_exposure(row):
        multiplier = symbol_multipliers.get(row['Symbol'], 1)
        s1 = row['Undl Price'] * multiplier
        s0 = row['Undl Price']
        k = row['Strike']
        return exposure(s1, s0, k)

    df['Exposure'] = df.apply(calculate_exposure, axis=1)

    owned = []
    exposure_minimum = 100000  # target exposure in raw units
    running_cost = 0

    dates = sorted(df['Date'].unique())
    for i, date in enumerate(dates):
        daily_df = df[df['Date'] == date].copy()

        # --- Step 1: Update DateDiff and recalc exposure for already owned contracts ---
        for contract in owned:
            contract['DateDiff'] -= 1  # decrement days to expiration
            symbol = contract['Symbol']
            daily_price_data = daily_df[daily_df['Symbol'] == symbol]
            if daily_price_data.empty:
                continue
            underlying_price_today = daily_price_data['Undl Price'].iloc[0]
            multiplier = symbol_multipliers.get(symbol, 1)
            s1 = underlying_price_today * multiplier
            s0 = underlying_price_today
            k = contract['Strike']
            contract['Exposure'] = exposure(s1, s0, k)

        # --- Step 2: Update value for owned contracts ---
        for contract in owned:
            matching_contract = daily_df[
                (daily_df['Symbol'] == contract['Symbol']) &
                (daily_df['Strike'] == contract['Strike']) &
                (daily_df['Maturity'] == contract['Maturity'])
            ]
            if not matching_contract.empty:
                ask_price_today = matching_contract['Ask Price'].iloc[0]
                exposure_today = contract['Exposure']
                contract['Value'] = find_value(
                    multiplier_list, contract['DateDiff'], ask_price_today, exposure_today
                )

        # --- Step 3: Process today's options: calculate exposure ---
        daily_df['Exposure'] = daily_df.apply(
            lambda row: exposure(
                row['Undl Price'] * symbol_multipliers.get(row['Symbol'], 1),
                row['Undl Price'],
                row['Strike']
            ), axis=1
        )

        # --- Step 4: Calculate today's value for new contracts ---
        daily_df['Value'] = daily_df.apply(
            lambda row: find_value(multiplier_list, row['DateDiff'], row['Ask Price'], row['Exposure']),
            axis=1
        )

        # --- Step 5: Sell all owned contracts (except on Day 1) ---
        # On days after the first, try to realize proceeds from owned contracts.
        if i > 0:
            for contract in owned:
                matching_contract = daily_df[
                    (daily_df['Symbol'] == contract['Symbol']) &
                    (daily_df['Strike'] == contract['Strike']) &
                    (daily_df['Maturity'] == contract['Maturity'])
                ]
                if not matching_contract.empty:
                    # If the contract has expired today, sell using the payoff.
                    if contract['Maturity'] <= date:
                        underlying_price_today = matching_contract['Undl Price'].iloc[0]
                        proceeds = call_payoff(underlying_price_today, contract['Strike'])
                    else:
                        # Otherwise, sell at the available bid price.
                        bid_price = matching_contract['Bid Price'].iloc[0]
                        proceeds = bid_price * (contract['Exposure'] * contract['Ask Size'])
                    running_cost -= proceeds

            # After selling, clear owned contracts.
            owned = []

        # --- Step 6: Purchase new contracts to meet the exposure requirement ---
        # Combine new and previously sold (if any) options.
        temp_df = pd.DataFrame(owned + daily_df.to_dict('records'))
        temp_df.sort_values('Value', ascending=False, inplace=True)

        current_exposure = sum(c['Exposure'] for c in owned)
        for _, option in temp_df.iterrows():
            if current_exposure >= exposure_minimum:
                break

            # Avoid duplicate purchases.
            exists = any(
                p['Symbol'] == option['Symbol'] and
                p['Strike'] == option['Strike'] and
                p['Maturity'] == option['Maturity']
                for p in owned
            )
            if exists:
                continue

            available = option['Ask Size'] * option['Exposure']
            # Purchase the full available contract.
            purchase = available

            new_contract = option.copy()
            new_contract['Exposure'] = purchase  # record full contract exposure purchased
            current_exposure += purchase
            running_cost += new_contract['Ask Price'] * (purchase / option['Exposure'])
            owned.append(new_contract.to_dict())

        #print(running_cost * 100, current_exposure, owned)

    # print(f"Running cost: {running_cost:.2f}, Exposure: {current_exposure:.2f}")
    return running_cost * 100

# Minimizer Function

Run the next two boxes. The first box minimizes from a random spot each time it loops, and it chooses the most optimal list. Starting from a random spot helps us prevent getting stuck in local minima. The second box prints out our best list, as well as the value we get from running loss_finder

In [45]:
# Box 1
max_date_diff = max(setup()['DateDiff']) + 1
bounds = [(-1, 10)] * max_date_diff

best_result = None
best_value = float('inf')

for _ in range(10):
    x0 = np.random.uniform(-1, 10, max_date_diff)
    result = minimize(loss_finder, x0, bounds=bounds, method='L-BFGS-B')

    if result.fun < best_value:
        best_value = result.fun
        best_result = result

In [61]:
# Box 2
optimal_list = best_result.x
optimal_val = best_result.fun
print(optimal_list, optimal_val)

[-0.88632001  1.32267596 -0.39418491  8.54997096  2.8661744   3.21422775
  8.71449322  4.54249698 -0.65326309  3.5920354   4.57878752  4.38489694] -19901499.93539986


Ignore boxes below!

In [ ]:
# mult_list = [b for b in range(11)]
mult_list = [1, 1, 1, 10, 1, 1, 1, 1, 10, 10, 10]
#mult_list = [2, 2, 2, 2, 8, 8, 8, 8, 8, 8, 8]
print(loss_finder(mult_list))

In [ ]:
def adjust_multipliers(mult_list, mult_index, spread): # This function will look at a single index of a multiplier list, adjust it up/down, and return which one gives us the cheapest execution 
    if mult_index == 0 or mult_index == 1: # We ignore index 0, and want to keep index 1 'anchored' so we don't have any sorts of runaway increase/decrease cycles 
        return mult_list 
    reg_mult_list = mult_list 
    if mult_list[mult_index] - spread < 0: # Negative numbers and zeroes will lead to problems so we filter out cases where this might happen 
         minus_return = 1000000000 
    else: # This pattern is followed 3 times below; first, we create a modified mult_list where only the ith index is changed, and then, we calculate the loss incurred by running the model with those weights. The times we run it are: reg_mult_list, plus_mult_list, and minus_mult_list 
        minus_mult_list = [mult_list[i] if i != mult_index else mult_list[i] - spread for i in range(len(mult_list))] 
        minus_return = loss_finder(minus_mult_list) 

    plus_mult_list = [mult_list[i] if i != mult_index else mult_list[i] + spread for i in range(len(mult_list))] 
    reg_return = loss_finder(reg_mult_list) 
    plus_return = loss_finder(plus_mult_list) 
    if min(reg_return, plus_return, minus_return) == reg_return: # Here, we return whichever list gave us the smallest loss 
        return reg_mult_list 
    elif min(reg_return, plus_return, minus_return) == minus_return: 
        return minus_mult_list 
    else: 
        return plus_mult_list 

def adjust_multipliers_once(mult_list, spread): # This function adjusts each index in our multipliers list once 
    temp_mult_list = mult_list 
    for i in range(len(mult_list)): # It runs adjust_multipliers on each index in the list 
        temp_mult_list = adjust_multipliers(temp_mult_list, i, spread) 
    return temp_mult_list # Returns our newly modified list 


q = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
change = 2
prev_value = 0
for _ in range(2): # We run an optimization 50 times 
    q = adjust_multipliers_once(q, change) # We adjust the multipliers in q 'change' amount 
    if prev_value == q: # If q doesn't change (ie, optimal given the change), make the change smaller 
        change /= 1.15
    print(loss_finder(q)) # For tracking how total loss evolves 
    prev_value = q
    print(q, change) # For tracking how the list evolves 